# MLOps Example with Iris Dataset

The **api.ifood.mlops** package is the Python SDK to interact with the apps of the ML platform: sandbox, database, pipeline and serving. It allows data scientists to develop and deploy ML models through experiments.

## 1. Setup

Let's start by importing packages, including the **api.ifood.mlops**.

In [ ]:
import json

import requests
import pandas as pd
from sklearn import *

from api.ifood import mlops

The first thing to do is to create a project! Let's create one called **iris** using the *create_project* method.

In [ ]:
project_id = mlops.create_project(name='iris')

print(project_id)

You can easily get all projects using the *get_projects* method.

In [ ]:
projects = mlops.get_projects()

for project in projects:
    for k, v in project.items():
        print(f"{k}: {v}")
    print("")

## 2. Experiment

Models can be trained and tested through experiments. Let's load the **iris** dataset from the simulated feature store and tweek a little bit with its data.

In [ ]:
dataset = pd.read_csv('./feature-store/iris.csv')

In [ ]:
dataset["target"] = dataset["class"].astype("category").cat.codes
dataset.drop("class", axis=1, inplace=True)
dataset.head()

Let's then divide the dataset into train and test datasets.

In [ ]:
train, test = model_selection.train_test_split(dataset, stratify=dataset["target"], random_state=42)

Now its time to create our firsts experiments using the *run_experiment* method. The method requires the following:

 - **project_id**: the id of the project we have just created;
 - **engine**: the Python ML engine to be used to develop the model (sklearn is the only available currently);
 - **model**: the engine model object initialized with its hyper params;
 - **metrics**: a dict with engine metrics functions;
 - **target_col**: the target column on the dataset;
 - **train_data**: the train dataset;
 - **test_data**: the test dataset.

We will create two experiments to predict the iris class: **Logistic Regression** model and **Perceptron** model, both will be assessed by their classification accuracy.

In [ ]:
lr_experiment_id = mlops.run_experiment(
    project_id=project_id,
    engine='sklearn',
    model=linear_model.LogisticRegression(),
    metrics=dict(accuracy=metrics.accuracy_score),
    target_col='target',
    train_data=train,
    test_data=test
)

print(lr_experiment_id)

In [ ]:
nn_experiment_id = mlops.run_experiment(
    project_id=project_id,
    engine='sklearn',
    model=linear_model.Perceptron(),
    metrics=dict(accuracy=metrics.accuracy_score),
    target_col='target',
    train_data=train,
    test_data=test
)

print(nn_experiment_id)

After running the *run_experiment* method, the **api.ifood.mlops** package will submit the request to the pipeline app. You can check out the experiment progress directly on the pipeline [dashboard](http://localhost:8080/) or using the *get_experiment* or *get_experiments* method. Run the cell bellow multiple times until both experiments status are equal to 'finished' then continue.

In [ ]:
experiments = mlops.get_experiments()

for experiment in experiments:
    for k, v in experiment.items():
        print(f"{k}: {v}")
    print("")

Let's compare them both:

In [ ]:
lr_experiment = mlops.get_experiment(experiment_id=lr_experiment_id)
print(f"LR status: {lr_experiment['status']}")
print(f"LR metric: {lr_experiment['metrics']}")

In [ ]:
nn_experiment = mlops.get_experiment(experiment_id=nn_experiment_id)
print(f"NN status: {nn_experiment['status']}")
print(f"NN metric: {nn_experiment['metrics']}")

Since the **Logistic Regression** accuracy is higher, lets deploy it!

## 3. Deploy

To the deploy an experiment, you can use the *deploy_experiment* method.

In [ ]:
mlops.deploy_experiment(experiment_id=lr_experiment_id)

After running the *deploy_experiment* method, the **api.ifood.mlops** package will submit the request to the pipeline app. You can check out the deployment progress directly on the pipeline [dashboard](http://localhost:8080/) or using the *get_experiment* or *get_experiments* method. Run the cell bellow multiple times until both experiments status are equal to 'deployed' then continue.

In [ ]:
lr_experiment = mlops.get_experiment(experiment_id=lr_experiment_id)
print(f"LR status: {lr_experiment['status']}")

You can check the deployed experiments with the *get_deployments* method.

In [ ]:
deployments = mlops.get_deployments()

for deployment in deployments:
    for k, v in deployment.items():
        print(f"{k}: {v}")
    print("")

## 4. Predict

Let's predict some cases! You can check the serving API docs [here](http://localhost:8000/docs).

 - Iris-setosa:

In [ ]:
data = dict(model="iris", features={"sepal-length": 5.7, "sepal-width": 3.8, "petal-length": 1.7, "petal-width": 0.3})

In [ ]:
try:
    response = requests.post(url='http://localhost:8000/predictions/', data=json.dumps(data), headers={'Content-Type': 'application/json', 'x-api-key': 'FfNxK6NF9L'})
    response.raise_for_status()
except Exception as exc:
    raise exc
else:
    print(response.text) # expecting "0" for Iris-setosa

 - Iris-versicolor:

In [ ]:
data = dict(model="iris", features={"sepal-length": 5.8, "sepal-width": 2.7, "petal-length": 4.1, "petal-width": 1.0})

In [ ]:
try:
    response = requests.post(url='http://localhost:8000/predictions/', data=json.dumps(data), headers={'Content-Type': 'application/json', 'x-api-key': 'FfNxK6NF9L'})
    response.raise_for_status()
except Exception as exc:
    raise exc
else:
    print(response.text) # expecting "1" for Iris-versicolor

 - Iris-virginica:

In [ ]:
data = dict(model="iris", features={"sepal-length": 7.7, "sepal-width": 3.0, "petal-length": 6.1, "petal-width": 2.3})

In [ ]:
try:
    response = requests.post(url='http://localhost:8000/predictions/', data=json.dumps(data), headers={'Content-Type': 'application/json', 'x-api-key': 'FfNxK6NF9L'})
    response.raise_for_status()
except Exception as exc:
    raise exc
else:
    print(response.text) # expecting "2" for Iris-virginica

You can get all the predictions for a project with the *get_predictions* method.

In [ ]:
predictions = mlops.get_predictions(project_id=project_id)

for prediction in predictions:
    for k, v in prediction.decode().items():
        print(f"{k}: {v}")
    print("")

## 5. Deploy a new model

The **Logistic Regression model** is outdated, let's deploy a **Support Vector Machine** model.

In [ ]:
svm_experiment_id = mlops.run_experiment(
    project_id=project_id,
    engine='sklearn',
    model=svm.SVC(),
    metrics=dict(accuracy=metrics.accuracy_score),
    target_col='target',
    train_data=train,
    test_data=test
)

print(svm_experiment_id)

Let's wait for the model status to be equals to 'finished'.

In [ ]:
experiment = mlops.get_experiment(experiment_id=svm_experiment_id)

for k, v in experiment.items():
    print(f"{k}: {v}")

Then deploy the model.

In [ ]:
mlops.deploy_experiment(experiment_id=svm_experiment_id)

Then predict data.

In [ ]:
data = dict(model="iris", features={"sepal-length": 5.7, "sepal-width": 3.8, "petal-length": 1.7, "petal-width": 0.3})

In [ ]:
try:
    response = requests.post(url='http://localhost:8000/predictions/', data=json.dumps(data), headers={'Content-Type': 'application/json', 'x-api-key': 'FfNxK6NF9L'})
    response.raise_for_status()
except Exception as exc:
    raise exc
else:
    print(response.text) # expecting "0" for Iris-setosa